Refer from
https://junstar92.tistory.com/105

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import h5py
import numpy as np
import math
def load_dataset():
train_dataset = h5py.File('train_signs.h5', "r")
train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels
test_dataset = h5py.File('test_signs.h5', "r")
test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels
classes = np.array(test_dataset["list_classes"][:]) # the list of classes
return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes
# Loading the data (signs)
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()
# Example of a picture
index = 6
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[index])))
X_train = X_train_orig.astype(np.float32)/255.
X_test = X_test_orig.astype(np.float32)/255.
Y_train = np.array(tf.one_hot(Y_train_orig, 6))
Y_test = np.array(tf.one_hot(Y_test_orig, 6))
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
m = X.shape[0]
mini_batches = []
np.random.seed(seed)
# Step 1: Shuffle (X, Y)
permutation = list(np.random.permutation(m))
shuffled_X = X[permutation,:,:,:]
shuffled_Y = Y[permutation,:]
# Step 2: Partition (shuffled_X, shuffled_Y)
num_complete_minibatches = math.floor(m/mini_batch_size)
for k in range(0, num_complete_minibatches):
mini_batch_X = shuffled_X[k * mini_batch_size:k * mini_batch_size + mini_batch_size,:,:,:]
mini_batch_Y = shuffled_Y[k * mini_batch_size:k * mini_batch_size + mini_batch_size,:]
mini_batch = (mini_batch_X, mini_batch_Y)
mini_batches.append(mini_batch)
# Handling the end case
if m % mini_batch_size != 0:
mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size:m,:,:,:]
mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size:m,:]
mini_batch = (mini_batch_X, mini_batch_Y)
mini_batches.append(mini_batch)
return mini_batchesb

In [ ]:
class ConvNet(tf.keras.Model):
def __init__(self):
super(ConvNet, self).__init__()
self.conv1 = tf.keras.layers.Conv2D(8, (4, 4), strides=(1,1), padding='same',
                                    activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))
self.maxpool1 = tf.keras.layers.MaxPool2D(pool_size=(8,8), strides=(8,8), padding='same')
self.conv2 = tf.keras.layers.Conv2D(16, (2, 2), strides=(1,1), padding='same',
                                    activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))
self.maxpool2 = tf.keras.layers.MaxPool2D(pool_size=(4,4), strides=(4,4), padding='same')
self.flatten = tf.keras.layers.Flatten()
self.d1 = tf.keras.layers.Dense(6, activation='softmax',
                                    kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))
def call(self, x):
x = self.conv1(x)
x = self.maxpool1(x)
x = self.conv2(x)
x = self.maxpool2(x)
x = self.flatten(x)
x = self.d1(x)
return x

In [ ]:
# Model
def Model_1(X_train, Y_train, X_test, Y_test, learning_rate=0.009,
    num_epochs=100, minibatch_size=64, print_cost=True):
    tf.random.set_seed(4)
    (m, n_H0, n_W0, n_C0) = X_train.shape
    seed = 3
    costs = []
    model = ConvNet()
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalCrossentropy(name='train_accuracy')
    for epoch in range(num_epochs):
        minibatch_cost=0
        num_minibatches = int(m/minibatch_size)
        seed = seed + 1
        minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
        for minibatch in minibatches:
            (minibatch_X, minibatch_Y) = minibatch
            with tf.GradientTape() as tape:
                pred = model(minibatch_X)
                cost = loss_object(minibatch_Y, pred)
            grads = tape.gradient(cost, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            train_loss(cost)
            train_accuracy(minibatch_Y, pred)
            minibatch_cost += cost / num_minibatches
        if print_cost and epoch % 5 == 0:
            print(f'Cost after epoch {epoch}: {minibatch_cost}')
        costs.append(minibatch_cost)
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    # training set
    y_pred = model(X_train)
    y_pred = tf.math.argmax(y_pred, 1)
    y_true = tf.math.argmax(Y_train, 1)
    train_acc = tf.math.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))
    y_pred = model(X_test)
    y_pred = tf.math.argmax(y_pred, 1)
    y_true = tf.math.argmax(Y_test, 1)
    test_acc = tf.math.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))
    print(f'Train acc : {train_acc}')
    print(f'Test acc : {test_acc}')
    return model

In [ ]:
m = Model_1(X_train, Y_train, X_test, Y_test)

In [ ]:
def Model_2(X_train, Y_train, X_test, Y_test, learning_rate=0.009,
num_epochs=100, minibatch_size=64):
    tf.random.set_seed(0)
    seed = 3
    (m, n_H0, n_W0, n_C0) = X_train.shape
    n_y = Y_train.shape[1]
    costs = []
    # initialize parameters
    #parameters = initialize_parameters()
    # setting optimizer
    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    # CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(8, (4, 4), strides=(1,1), padding="SAME",
                                activation='relu', input_shape=(64, 64, 3),
                                kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0)),
        tf.keras.layers.MaxPool2D(pool_size=(8,8), strides=(8,8), padding="SAME"),
        tf.keras.layers.Conv2D(16, (2, 2), strides=(1,1), padding="SAME", activation='relu',
                                kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0)),
        tf.keras.layers.MaxPool2D(pool_size=(4,4), strides=(4,4), padding="SAME"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(6, activation='softmax',
                                kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))
    ])
    model.summary()
    model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    hist = model.fit(X_train, Y_train, epochs=num_epochs, batch_size=minibatch_size, verbose=1)
    costs = hist.history['loss']
    for epoch in range(0, num_epochs, 5):
        print(f'Cost after epoch {epoch}: {costs[epoch]}')
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    train_acc = hist.history['accuracy'][-1]
    y_pred = model.predict(X_test)
    y_pred = tf.math.argmax(y_pred, 1)
    y_true = tf.math.argmax(Y_test, 1)
    test_acc = tf.math.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))
    print(f'Train acc : {train_acc}')
    print(f'Test acc : {test_acc}')
    return model

In [ ]:
m2 = Model_2(X_train, Y_train, X_test, Y_test)

In [ ]:
def Model_2_Reg(X_train, Y_train, X_test, Y_test, learning_rate=0.009,
num_epochs=100, minibatch_size=64, lambd=0.01):
    tf.random.set_seed(2)
    seed = 3
    (m, n_H0, n_W0, n_C0) = X_train.shape
    n_y = Y_train.shape[1]
    costs = []
    # initialize parameters
    #parameters = initialize_parameters()
    # setting optimizer
    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    # setting regularizer
    regularizer = tf.keras.regularizers.L2(lambd)
    # CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(8, (4, 4), strides=(1,1), padding="SAME",
                                activation='relu', input_shape=(64, 64, 3),
                                kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0),
                                kernel_regularizer=regularizer),
        tf.keras.layers.MaxPool2D(pool_size=(8,8), strides=(8,8), padding="SAME"),
        tf.keras.layers.Conv2D(16, (2, 2), strides=(1,1), padding="SAME", activation='relu',
                                kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0),
                                kernel_regularizer=regularizer),
        tf.keras.layers.MaxPool2D(pool_size=(4,4), strides=(4,4), padding="SAME"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(6, activation='softmax',
                                kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0),
                                kernel_regularizer=regularizer)
    ])
    model.summary()
    model.compile(optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'])
    hist = model.fit(X_train, Y_train, epochs=num_epochs, batch_size=minibatch_size, verbose=1)
    costs = hist.history['loss']
    for epoch in range(0, num_epochs, 5):
        print(f'Cost after epoch {epoch}: {costs[epoch]}')
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    train_acc = hist.history['accuracy'][-1]
    y_pred = model.predict(X_test)
    y_pred = tf.math.argmax(y_pred, 1)
    y_true = tf.math.argmax(Y_test, 1)
    test_acc = tf.math.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))
    print(f'Train acc : {train_acc}')
    print(f'Test acc : {test_acc}')
    return model